In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from matplotlib_venn import venn3, venn2
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper
import train
from sklearn import decomposition
from sklearn.calibration import calibration_curve

outcome_def = 'y_consec_thresh'
out_dir = 'results/jan9_4'
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models
**baseline stats**

In [15]:
metadata_file = 'processed/metadata.pkl'
m = pkl.load(open(metadata_file, 'rb'))

print(f'valid:\t\t{m["num_aux_pos_after_peak_time"]} aux+ / {m["num_tracks_after_peak_time"]} ({m["num_aux_pos_after_peak_time"]/m["num_tracks_after_peak_time"]:.3f})')
print(f'lifetime<={m["thresh_short"]}:\t{round(m["num_short"] * m["acc_short"]):>4.0f} aux+ / {m["num_short"]} ({m["acc_short"]:.3f})')
print(f'lifetime>={m["thresh_long"]}:\t{round(m["num_long"] * m["acc_long"]):>4.0f} aux- / {m["num_long"]:>4} ({m["acc_long"]:.3f})')
print(f'remaining:\t{m["num_aux_pos_after_lifetime"]} aux+ / {m["num_tracks_after_lifetime"]} ({m["num_aux_pos_after_lifetime"]/m["num_tracks_after_lifetime"]:.3f})')

valid:		1702 aux+ / 6372 (0.267)
lifetime<=25:	3802 aux+ / 4145 (0.917)
lifetime>=135:	 130 aux- /  140 (0.929)
remaining:	1229 aux+ / 2087 (0.589)


**look at prediction metrics**

In [ ]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

**look at feat importances**

In [48]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r[r.accuracy > 0.65]
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('39')]
r = r[~r.index.str.contains('=11')]
r = r[~r.index.str.contains('=7')]
r = r[~r.index.str.contains('=15')]
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('11')]
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)

subset = list(r.keys())
subset.remove('acc')
r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored

,acc,fall,X_mean,X_min,rise,X_max,sc_4,center_max,d2x_max_spl,down_max,dx_max_spl,left_max,lifetime,max_diff,msd,nmf_11,nmf_3,pc_2,pc_6,right_max,rise_local_3,sc_0,sc_1,td,up_max
model_type,,,,,,,,,,,,,,,,,,,,,,,,,
mlp2_15_ros=1_select_rf=5,0.677,2,1,0,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mlp2_25_ros=1_select_rf=3,0.671,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mlp2_15_ros=1_select_rf=3,0.667,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mlp2_25_ros=1_select_rf=5,0.666,1,0,3,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# analyze model predictions

In [ ]:
df = data_tracks.get_data()
n = df.shape[0]

# normalize and store
X = df[data_tracks.get_feature_names(df)]
X_mean = X.mean()
X_std = X.std()
ks = list(X.keys())
norms = {ks[i]: {'mu': X_mean[i], 'std': X_std[i]} for i in range(len(ks))}
X = (X - X_mean) / X_std
y = df[outcome_def].values


# split testing data based on cell num
# cv_idx = data_tracks.cell_nums_train[0]
# idxs_test = df.cell_num.isin([cv_idx]) # this is the cv set for model's trained in the first cv fold
# X_test, Y_test = X[idxs_test], y[idxs_test]

# look at best models
r = results
r = r.sort_values('accuracy', ascending=False)
idx = np.array(r.index)
print(idx[:10])

### look at single model

In [17]:
model_name = idx[0]

In [18]:
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_name, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values
preds = d_full_cv[model_name].values
preds_proba = d_full_cv[model_name + '_proba'].values

results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
assert np.mean(preds==y_full_cv) == np.average(results_individual['cv']['accuracy'], weights=results_individual['num_pts_by_fold_cv']), 'did not properly load model/data'

In [ ]:
viz.viz_errs_1d(X.iloc[idxs_cv], preds, preds_proba, y_full_cv, norms, key='lifetime')

In [ ]:
viz.viz_errs_1d(X.iloc[idxs_cv], preds, preds_proba, y_full_cv, norms, key='X_min')

In [ ]:
viz.viz_errs_1d(X.iloc[idxs_cv], preds, preds_proba, y_full_cv, norms, key='fall')

In [ ]:
viz.viz_errs_2d(df, idxs_cv, preds, y_full_cv, key1='x_pos', key2='y_pos')

In [ ]:
viz.viz_errs_2d(df, idxs_cv, preds, y_full_cv, key1='fall', key2='X_min')

In [ ]:
viz.viz_errs_2d(df, idxs_cv, preds, y_full_cv, key1='pc1', key2='pc2', X=X)

In [ ]:
viz.viz_errs_outliers(X.iloc[idxs_cv], preds, y_full_cv, num_feats_reduced=5)

In [ ]:
idxs_fp = preds > y_full_cv
idxs_fn = preds < y_full_cv
viz.viz_biggest_errs(df, df['X'].iloc[idxs_cv][idxs_fp], df['Y'].iloc[idxs_cv][idxs_fp], y_full_cv[idxs_fp], preds[idxs_fp], preds_proba[idxs_fp])    
viz.viz_biggest_errs(df, df['X'].iloc[idxs_cv][idxs_fn], df['Y'].iloc[idxs_cv][idxs_fn], y_full_cv[idxs_fn], preds[idxs_fn], preds_proba[idxs_fn])    

In [ ]:
plt.figure(dpi=200)
prob_true, prob_pred = calibration_curve(y_full_cv, preds_proba, normalize=False, n_bins=10, strategy='quantile')
plt.plot(prob_true, prob_pred, '.')
plt.plot([0, 1], [0, 1], alpha=0.3, color='gray')
plt.xlabel('true prob')
plt.ylabel('predicted prob')
plt.show()

**look at pcs**

In [ ]:
# perform pca
pca = decomposition.PCA(whiten=True)
pca.fit(X.iloc[idxs_cv])
viz.plot_pcs(pca, X.iloc[idxs_cv])

In [ ]:
pca = decomposition.PCA(n_components=2, whiten=True)
X_reduced = pca.fit_transform(X.iloc[idxs_cv])
plt.figure(dpi=200)
ys_neg = y[idxs_cv] == 0
plt.plot(X_reduced[:, 0][ys_neg], X_reduced[:, 1][ys_neg], 'o', 
         color=cr, alpha=0.3, markeredgewidth=0, ms=2)
plt.plot(X_reduced[:, 0][~ys_neg], X_reduced[:, 1][~ys_neg], 'o', 
         color=cb, alpha=0.3, markeredgewidth=0, ms=2)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

### look at many models

In [95]:
# model_names = ['mlp2_11_none', 'svm_35_none', 'logistic_4_none', 'rf_9_none']
# model_names = ['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']
model_names = idx[:3]
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_names, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values

**venn-diagram**

In [ ]:
sets = []
for model_name in model_names:
    args = np.argwhere(d_full_cv[model_name] != y_full_cv)
    sets.append(set(args.flatten().tolist()))
    
plt.figure(dpi=300)
plt.title('venn diagram of shared errors')
venn3(sets, model_names[:3])
plt.show()

**ensemble err**

In [97]:
# ensemble
d_full_cv_probs = d_full_cv[[k for k in d_full_cv.keys() if 'proba' in k]]
preds_soft = d_full_cv_probs.sum(axis=1) / d_full_cv_probs.shape[1]

for score_name in train.scorers.keys():
    print(score_name, train.scorers[score_name](y_full_cv, preds_soft > 0.5))
# metrics.accuracy_score(y_ensemble, preds_soft>0.5) 

balanced_accuracy 0.6792986425339367
accuracy 0.6892545982575025
precision 0.7924217462932455
recall 0.7115384615384616
f1 0.7498051441932968
roc_auc 0.6792986425339367
precision_recall_curve (array([0.65440465, 0.79242175, 1.        ]), array([1.        , 0.71153846, 0.        ]), array([False,  True]))
roc_curve (array([0.        , 0.35294118, 1.        ]), array([0.        , 0.71153846, 1.        ]), array([2, 1, 0]))
